In [1]:
import os
import wandb
import random
import torch
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, ControlNetModel
from transformers import AutoTokenizer, PretrainedConfig
from transformers import CLIPTextModel

import torch.nn.functional as F
os.environ['HF_HOME'] = 'hf_cache'

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0.post300 with CUDA 1102 (you have 2.2.2+cu121)
    Python  3.11.6 (you have 3.11.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
if torch.cuda.is_available():
    # Set device to GPU
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    # Set device to CPU
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU.


In [ ]:
def import_model_class_from_model_name_or_path(pretrained_model_name_or_path: str, revision: str):
    text_encoder_config = PretrainedConfig.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="text_encoder",
        revision=revision,
    )
    model_class = text_encoder_config.architectures[0]

    if model_class == "CLIPTextModel":
        from transformers import CLIPTextModel

        return CLIPTextModel
    elif model_class == "RobertaSeriesModelWithTransformation":
        from diffusers.pipelines.alt_diffusion.modeling_roberta_series import RobertaSeriesModelWithTransformation

        return RobertaSeriesModelWithTransformation
    else:
        raise ValueError(f"{model_class} is not supported.")

In [34]:
from transformers import CLIPTokenizer, CLIPTextModel

In [35]:
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [62]:
inputs = tokenizer([""]*5,  padding=True, return_tensors="pt")

In [63]:
inputs

{'input_ids': tensor([[49406, 49407],
        [49406, 49407],
        [49406, 49407],
        [49406, 49407],
        [49406, 49407]]), 'attention_mask': tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]])}

In [64]:
outputs = model(**inputs)

In [65]:
outputs[0].shape

torch.Size([5, 2, 512])

In [51]:
timesteps = torch.randint(0, 1000, (5,), device=latents.device)
timesteps = timesteps.long()

In [52]:
timesteps

tensor([257, 551,  38, 280,  82])

In [54]:
latents = torch.zeros((5, 3, 128, 128), dtype = torch.float16).to(device)
latents.shape

torch.Size([5, 3, 128, 128])

In [56]:
noisy_latents = vae.encoder(latents)
noisy_latents.shape

torch.Size([5, 8, 32, 32])

In [66]:
unet(noisy_latents, timesteps.to(device), encoder_hidden_states=outputs[0].to(device))

ValueError: class_labels should be provided when num_class_embeds > 0

In [8]:
vae = AutoencoderKL.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", 
    subfolder="vae",
    torch_dtype=torch.float16,
    use_safetensors = True
)

In [24]:
unet = UNet2DConditionModel.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", 
    subfolder="unet", 
    torch_dtype=torch.float16, 
    use_safetensors = True
)

In [ ]:
noise_scheduler = DDPMScheduler.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", 
    subfolder="scheduler"
)

In [28]:
text_encoder_cls = CLIPTextModel
text_encoder = text_encoder_cls.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler", 
    subfolder="text_encoder", 
    torch_dtype=torch.float16, 
    use_safetensors = True
)

In [ ]:
controlnet = ControlNetModel.from_unet(unet)

In [ ]:
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)
controlnet.train()

In [14]:
vae.requires_grad_(False)

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [15]:
vae.to(device)

AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [29]:
text_encoder.to(device)

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1024)
      (position_embedding): Embedding(77, 1024)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-22): 23 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1024,

In [31]:
img = torch.zeros([5, 3, 128, 128], dtype=torch.float16).to(device)
img.shape

torch.Size([5, 3, 128, 128])

In [32]:
en_out = vae.encoder(img)
en_out.shape

torch.Size([5, 8, 32, 32])

In [25]:
unet.to(device)

UNet2DConditionModel(
  (conv_in): Conv2d(7, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=256, out_features=1024, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (class_embedding): Embedding(1000, 1024)
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 256, eps=1e-05, affine=True)
          (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=1024, out_features=256, bias=True)
          (norm2): GroupNorm(32, 256, eps=1e-05, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList

In [27]:
unet(en_out, )

TypeError: UNet2DConditionModel.forward() missing 2 required positional arguments: 'timestep' and 'encoder_hidden_states'

In [ ]:
timesteps = torch.randint(0, 1000, (bsz,), device=latents.device)
timesteps = timesteps.long()

In [33]:
vae.decoder(en_out)

RuntimeError: Given groups=1, weight of size [512, 4, 3, 3], expected input[5, 8, 32, 32] to have 4 channels, but got 8 channels instead